In [1]:
import os
import numpy as np
import mido
from vapory import *
from moviepy.editor import VideoClip, AudioFileClip

In [2]:
MIDI_NAME = "../songs/guren.mid"
MP3_NAME = MIDI_NAME[:-3]+"mp3"
VIDEO_NAME = MIDI_NAME[:-3]+"mp4"

In [3]:
# os.system('fluidsynth piano.SF2 %s -T raw -F - | lame -r - "%s"'%(
#            MIDI_NAME, MP3_NAME))

In [4]:
midi = mido.MidiFile(MIDI_NAME)

In [5]:
info_track = midi.tracks[0]
piano_track = midi.tracks[1]

In [6]:
tempo = [e.tempo for e in info_track if hasattr(e,'tempo')][0]

In [7]:
seconds_per_beat = 1.0 * tempo / 10**6 
seconds_per_tick = seconds_per_beat / midi.ticks_per_beat

In [8]:
events = {note: [] for note in range(0,120)}

In [9]:
events

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [],
 55: [],
 56: [],
 57: [],
 58: [],
 59: [],
 60: [],
 61: [],
 62: [],
 63: [],
 64: [],
 65: [],
 66: [],
 67: [],
 68: [],
 69: [],
 70: [],
 71: [],
 72: [],
 73: [],
 74: [],
 75: [],
 76: [],
 77: [],
 78: [],
 79: [],
 80: [],
 81: [],
 82: [],
 83: [],
 84: [],
 85: [],
 86: [],
 87: [],
 88: [],
 89: [],
 90: [],
 91: [],
 92: [],
 93: [],
 94: [],
 95: [],
 96: [],
 97: [],
 98: [],
 99: [],
 100: [],
 101: [],
 102: [],
 103: [],
 104: [],
 105: [],
 106: [],
 107: [],
 108: [],
 109: [],
 110: [],


In [10]:
t=0
for e in piano_track:
    t += seconds_per_tick * e.time
    if e.type in ["note_on", "note_off"]:
        events[e.note].append((t, e.type, e.velocity))

In [11]:
events

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [],
 15: [],
 16: [],
 17: [],
 18: [],
 19: [],
 20: [],
 21: [],
 22: [],
 23: [],
 24: [],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [],
 35: [],
 36: [],
 37: [],
 38: [],
 39: [],
 40: [],
 41: [],
 42: [],
 43: [],
 44: [],
 45: [],
 46: [],
 47: [],
 48: [],
 49: [],
 50: [],
 51: [],
 52: [],
 53: [],
 54: [(21.216869216666655, 'note_on', 93),
  (21.29283613333332, 'note_off', 0),
  (24.372258691666655, 'note_on', 80),
  (24.503474274999988, 'note_off', 0),
  (24.702369474999987, 'note_on', 88),
  (25.029027216666652, 'note_off', 0),
  (25.029027216666652, 'note_on', 88),
  (25.043529991666652, 'note_off', 0),
  (27.354305474999986, 'note_on', 88),
  (27.68096321666665, 'note_off', 0),
  (27.68096321666665, 'note_on', 88),
  (27.69546599166665, 'note_off', 0),
  (29.67613069166665, 'note_on', 80),
  (29.807346274999

In [12]:
def get_height(note, t):
    """ Returns the height of a given key at the given time. """

    slope = lambda t : max(0,min(1,t))

    def term(t,event):
        event_time, event_type, velocity = event
        if event_type == 'note_on':
            return -slope(100*(t-event_time))
        else:
            return +slope(100*(t-event_time))
    
    return max(0,min(1,1+sum([ term(t,e) for e in events[note]])))

In [13]:
# Some dimensions of the piano keys and the black "piano box"

key_w = 0.4
key_h = 1.1 * key_w
key_l = 5 * key_w
box_bottom = 0.2

In [14]:
light = LightSource([0, 1000, -1000], [1.4,1.4,1.4],'parallel','point_at',[0,0,0])
wall = Plane([0,0,1],20, Texture(Pigment('color', [1,1,1])))

ground = Plane( [0,1,0], 0, Texture( Pigment( 'color', [1,1,1]),
                                     Finish( 'phong', 0.1,
                                             'reflection',0.2,
                                             'metallic', 0.3)))

piano_box = Box([-60*key_w, 0,1-key_l-0.1], [60*key_w,box_bottom, 1.01],
                Texture( Pigment( 'color', [0,0,0]), Finish('reflection', 0.0)))

camera = Camera("location", [0, 15, -30], "look_at", [0, 0, 0], "angle", 75)
scene = Scene( camera, objects = [ ground, light, piano_box])

In [15]:
# COMPUTE THE COORDINATES AND COLOR OF THE DIFFERENT KEYS

def compute_key_attributes(key):
    distance_to_C4 = key-60
    octave, note = distance_to_C4//12, distance_to_C4%12
    x_coordinate = 14*octave + note + (note>4)
    color = 'black' if (note in [1,3,6,8,10]) else 'white'
    return (x_coordinate, color)

In [16]:
keys = range(21, 108)
keys_attributes = { k: compute_key_attributes(k) for k in keys}

In [17]:
def make_piano_keys(keys_heights):
    keys = []
    for key, height in keys_heights.items():
        x_c, color = keys_attributes[key]  
        x_c = x_c*(key_w*1.03)-2
        
        if color=='white':
            w, h, l, color = key_w, key_h*(1-0.7*(1-height)), key_l, [1,1,1]
        else:
            w, h, l, color = (0.47*key_w, key_h*(1.5-0.4*(1-height)),
                              0.6*key_l, [0,0,0])
        keys.append(Box([x_c-w, box_bottom,1-l], [x_c+w,box_bottom+h,1],
                        Texture( Pigment( 'color', color),
                                 Finish('ambient', 0.2, 'brilliance',0.3))))
    return keys

In [18]:
# WE KNOW DEFINE EXACTLY THE FRAME AT TIME t, AND ASSEMBLE THE VIDEO.

def make_frame(t):
    keys_heights = {k: get_height(k,t) for k in keys}
    newscene = scene.add_objects(make_piano_keys(keys_heights))
    newscene.camera = Camera( "location", [0, 10, -25], "look_at", [0, 0, 0],
                              "angle", 70)
    return newscene.render(width=1000, height=100, antialiasing=0.001)

In [19]:
audio = AudioFileClip(MP3_NAME)

In [20]:
clip = VideoClip(make_frame, duration = audio.duration).subclip(.05)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
clip.write_videofile(VIDEO_NAME, fps=30, bitrate='8000k', audio=MP3_NAME)